We begin by loading some of the necessary libraries!

In [186]:
import numpy as np
import pandas as pd

import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

print('Libraries imported')


Libraries imported


To scrape the data, I saved the web page as a pdf and then copied and pasted it into excel to create a .csv file that could be inserted into the code to create the data frame. 

In [150]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Borough,Neighborhood
0,M4N,Central Toronto,Lawrence Park
1,M5N,Central Toronto,Roselawn
2,M4P,Central Toronto,Davisville North
3,M5P,Central Toronto,Forest Hill North & West
4,M4R,Central Toronto,North Toronto West


In [151]:
postal_df.tail()

,Postal Code,Borough,Neighborhood
98,M6C,York,Humewood-Cedarvale
99,M6E,York,Caledonia-Fairbanks
100,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn"
101,M6N,York,"Runnymede, The Junction North"
102,M9N,York,Weston


I have already eliminated the 'not assigned' entries from the csv before inserting it in the code

In [152]:
postal_df.shape

(103, 3)

Now I create a data frame from the merged csv file to add the Latitude and Longitude columns.

In [153]:
body = client_77c71a8d289e4e20ac6fa5cff69e0829.get_object(Bucket='torontoneighborhoodclustering-donotdelete-pr-ugjoe5uabjsj30',Key='TorontoPostalCodeswithLL.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
geopostal_df = pd.read_csv(body)
geopostal_df.head()


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [154]:
geopostal_df.tail()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
102,M9W,Etobicoke,Northwest,43.706748,-79.594054


In [155]:
geopostal_df.shape

(103, 5)

The combined data frame has the same 103 rows with two more columns than before

In [1]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

In [6]:
get_ipython().system('pip install folium==0.5.0') 
import folium

     |████████████████████████████████| 81kB 7.2MB/s eta 0:00:011
  Stored in directory: /home/dsxuser/.cache/pip/wheels/f8/98/ff/954791afc47740d554f0d9e5885fa09dd60c2265d42578e665
Successfully built folium


Now we create a map of Toronto and superimpose the boroughs on top

In [156]:
address='Toronto, Canada'
geolocator=Nominatim(user_agent="toronto_explorer")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print ('The geographical coordinates of Toronto are {},{}.'.format(latitude,longitude))

The geographical coordinates of Toronto are 43.6534817,-79.3839347.


In [187]:
map_toronto=folium.Map(location=[latitude,longitude],zoom_start=10)

for lat,lng,borough,neighborhood in zip(geopostal_df['Latitude'],geopostal_df['Longitude'],geopostal_df['Borough'],geopostal_df['Neighborhood']):
    label='{},{}'.format(neighborhood,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radium=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

Now we will create a more limited data set that only includes neighborhoods in downtown Toronto

In [188]:
body = client_77c71a8d289e4e20ac6fa5cff69e0829.get_object(Bucket='torontoneighborhoodclustering-donotdelete-pr-ugjoe5uabjsj30',Key='TorontoNames.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
Toronto_df1 = pd.read_csv(body)
Toronto_df1.head(30)
Toronto_df=Toronto_df1[9:28]
Toronto_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
9,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
10,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
11,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
12,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
14,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
15,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
16,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
17,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
18,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752


In [189]:
Toronto_df.shape

(19, 5)

In [191]:
map_limit=folium.Map(location=[latitude,longitude],zoom_start=10)

for lat,lng,borough,neighborhood in zip(Toronto_df['Latitude'],Toronto_df['Longitude'],Toronto_df['Borough'],Toronto_df['Neighborhood']):
    label='{},{}'.format(neighborhood,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radium=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_limit)
    
map_limit

Next I will explore downtown Toronto 

In [192]:
CLIENT_ID = '1GF2ZURUPIUCMKFFK2JWUB34Z1CSFXO2GFTJ2BIGJMQIJRE2'
CLIENT_SECRET ='044X1QMU5HZGZCF1LHDCJCLL5ZY0VH1GL5HUYSIBMKQZSW55'
VERSION = '20200518'

First for just one neighborhood on the list

In [193]:
Toronto_df.loc[9,'Neighborhood']

'Rosedale'

In [194]:
neighborhood_latitude=Toronto_df.loc[9,'Latitude']
neighborhood_longitude=Toronto_df.loc[9,'Longitude']
neighborhood_name=Toronto_df.loc[9,'Neighborhood']

In [195]:
LIMIT=30
radius=500
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighborhood_latitude,
    neighborhood_longitude,
    radius,
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=1GF2ZURUPIUCMKFFK2JWUB34Z1CSFXO2GFTJ2BIGJMQIJRE2&client_secret=044X1QMU5HZGZCF1LHDCJCLL5ZY0VH1GL5HUYSIBMKQZSW55&v=20200518&ll=43.6795626,-79.37752940000001&radius=500&limit=30'

In [196]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [134]:
results =requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ec33578be61c9001b43e2be'},
 'response': {'headerLocation': 'Rosedale',
  'headerFullLocation': 'Rosedale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.6840626045, 'lng': -79.37131878274371},
   'sw': {'lat': 43.675062595499995, 'lng': -79.38374001725632}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4aff2d47f964a520743522e3',
       'name': 'Rosedale Park',
       'location': {'address': '38 Scholfield Ave.',
        'crossStreet': 'at Edgar Ave.',
        'lat': 43.68232820227814,
        'lng': -79.37893434347683,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.68232820227814,
          'lng': -79.37893434347683}],
        'distance': 32

In [197]:

def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [166]:
venues = results['response']['groups'][0]['items']
nearby_venue = json_normalize(venues)
filtered_columns = ['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues =nearby_venue.loc[:,filtered_columns]
nearby_venues['venue.categories']=nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()


,name,categories,lat,lng
0,Rosedale Park,Playground,43.682328,-79.378934
1,Whitney Park,Park,43.682036,-79.373788
2,Alex Murray Parkette,Park,43.678300,-79.382773
3,Milkman's Lane,Trail,43.676352,-79.373842


In [202]:
def getNearbyVenues(names,latitudes,longitudes,radius=500):
    
    venues_list=[]
    for name,lat,lng in zip(names,latitudes,longitudes):
        print(name)
       
        url1='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)

        results=requests.get(url1).json()["response"]['groups'][0]['items']

        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns =['Neighborhood',
                    'Neighborhood Latitude',
                    'Neighborhood Longitude',
                    'Venue',
                    'Venue Latitude',
                    'Venue Longitude',
                    'Venue Category']

    return(nearby_venues)
   

In [203]:
downtown_venues=getNearbyVenues(names=Toronto_df['Neighborhood'],
                               latitudes=Toronto_df['Latitude'],
                               longitudes=Toronto_df['Longitude']
                               )

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government


In [204]:
print(downtown_venues.shape)
downtown_venues

(516, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"St. James Town, Cabbagetown",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner
5,"St. James Town, Cabbagetown",43.667967,-79.367675,F'Amelia,43.667536,-79.368613,Italian Restaurant
6,"St. James Town, Cabbagetown",43.667967,-79.367675,Butter Chicken Factory,43.667072,-79.369184,Indian Restaurant
7,"St. James Town, Cabbagetown",43.667967,-79.367675,Kingyo Toronto,43.665895,-79.368415,Japanese Restaurant
8,"St. James Town, Cabbagetown",43.667967,-79.367675,Murgatroid,43.667381,-79.369311,Restaurant
9,"St. James Town, Cabbagetown",43.667967,-79.367675,Merryberry Cafe + Bistro,43.666630,-79.368792,Café


In [205]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,30,30,30,30,30,30
Christie,17,17,17,17,17,17
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
"First Canadian Place, Underground city",30,30,30,30,30,30
"Garden District, Ryerson",30,30,30,30,30,30
"Harbourfront East, Union Station, Toronto Islands",30,30,30,30,30,30


In [206]:
print('There are {} unique categories'.format(len(downtown_venues['Venue Category'].unique())))

There are 148 unique categories


Now we analyse each neighborhood in preparation for clustering

In [207]:
downtown_onehot=pd.get_dummies(downtown_venues[['Venue Category']],prefix="",prefix_sep="")
downtown_onehot['Neighborhood']=downtown_venues['Neighborhood']
fixed_columns=[downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]
downtown_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [208]:
downtown_onehot.shape

(516, 148)

Now we group by neighborhood

In [226]:
downtown_grouped=downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.033333,0.000000,0.000000,0.00,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.066667,0.066667,0.066667,0.066667,0.2,0.133333,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Central Bay Street,0.033333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.033333,0.000000,0.000000,0.00,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.033333,0.000000,0.033333,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.033333,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,"First Canadian Place, Underground city",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.033333,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
7,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.033333,0.066667,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.033333,...,0.000000,0.000000,0.000000,0.00,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.033333,0.000000,0.066667,0.033333,0.000000


In [225]:
downtown_grouped.shape

(19, 148)

Now we print a list of each neighborhood with the top 5 most common venues in the neighborhood

In [227]:
num_top_venues=5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp=downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns=['venue','freq']
    temp=temp.iloc[1:]
    temp['freq']=temp['freq'].astype(float)
    temp=temp.round({'freq':2})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0            Beer Bar  0.07
1  Seafood Restaurant  0.07
2         Coffee Shop  0.07
3        Cocktail Bar  0.07
4              Bistro  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0      Airport Service  0.20
1     Airport Terminal  0.13
2  Rental Car Location  0.07
3                Plane  0.07
4     Sculpture Garden  0.07


----Central Bay Street----
             venue  freq
0      Coffee Shop  0.23
1             Café  0.07
2      Yoga Studio  0.03
3             Park  0.03
4  Bubble Tea Shop  0.03


----Christie----
                venue  freq
0       Grocery Store  0.24
1                Café  0.18
2                Park  0.12
3  Athletics & Sports  0.06
4          Restaurant  0.06


----Church and Wellesley----
             venue  freq
0      Coffee Shop  0.03
1       Smoke Shop  0.03
2   Breakfast Spot  0.03
3  Bubble Tea Shop  0.03

Now that goes into a dataframe

In [270]:
def return_most_common_venues(row,num_top_venues):
    row_categories =row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [271]:
num_top_venues =10
indicators=['st','nd','rd']
columns=['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1,indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
neighborhoods_venues_sorted=pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood']=downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind,1:]=return_most_common_venues(downtown_grouped.iloc[ind,:], num_top_venues)
    
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Beer Bar,Liquor Store,Bistro,Creperie,Restaurant,Breakfast Spot,Bakery
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Plane,Boutique,Rental Car Location,Sculpture Garden,Harbor / Marina,Boat or Ferry,Airport Lounge,Airport Gate
2,Central Bay Street,Coffee Shop,Café,Spa,Sandwich Place,Ramen Restaurant,Poke Place,Bubble Tea Shop,Park,Modern European Restaurant,Middle Eastern Restaurant
3,Christie,Grocery Store,Café,Park,Italian Restaurant,Coffee Shop,Candy Store,Nightclub,Restaurant,Baby Store,Athletics & Sports
4,Church and Wellesley,Coffee Shop,Pub,Bookstore,Juice Bar,Restaurant,Breakfast Spot,Salon / Barbershop,Japanese Restaurant,Beer Bar,Bubble Tea Shop


We will try 5 clusters

In [272]:
kclusters=5
downtown_grouped_clustering=downtown_grouped.drop('Neighborhood',1)
kmeans=KMeans(n_clusters=kclusters,random_state=0).fit(downtown_grouped_clustering)
kmeans.labels_[0:10]

array([2, 4, 0, 3, 2, 0, 0, 0, 2, 2], dtype=int32)

Now we can add the clusters to the dataframe

In [273]:
neighborhoods_venues_sorted.insert(0, 'Cluster No.',kmeans.labels_)
downtown_merged=Toronto_df
downtown_merged=downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'),on='Neighborhood')
downtown_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster No.,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Colombian Restaurant,Dance Studio,Creperie,Cosmetics Shop,Concert Hall,Comic Shop,Comfort Food Restaurant
10,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,2,Coffee Shop,Café,Italian Restaurant,Restaurant,Bakery,Playground,Butcher,Pub,Deli / Bodega,Caribbean Restaurant
11,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,2,Coffee Shop,Pub,Bookstore,Juice Bar,Restaurant,Breakfast Spot,Salon / Barbershop,Japanese Restaurant,Beer Bar,Bubble Tea Shop
12,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Breakfast Spot,Theater,Bakery,Ice Cream Shop,Performing Arts Venue,Café,Pub,Restaurant
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Café,Theater,Coffee Shop,Music Venue,Steakhouse,Comic Shop,Ramen Restaurant,College Rec Center,Sandwich Place,Plaza


Now we can visualize the clusters

In [275]:
map_clusters=folium.Map(location=[latitude,longitude],zoom_start=11)
x=np.arange(kclusters)
ys=[i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat,lon,poi,cluster in zip(downtown_merged['Latitude'],downtown_merged['Longitude'],downtown_merged['Neighborhood'],downtown_merged['Cluster No.']):
    label = folium.Popup(str(poi) + 'Cluster' + str(cluster),parse_html=True)
    folium.CircleMarker(
    [lat,lon],
    radius=5,
    popup=label,
    color=rainbow[cluster-1],
    fill=True,
    fill_color=rainbow[cluster-1],
    fill_opacity=0.7).add_to(map_clusters)
map_clusters
    

Turns out most of the downtown neighborhoods are similar in terms of types of venues, with the three neighborhoods that are more on the outskirts falling into clusters of 1